In [1]:
import numpy as np
import pandas as pd

In [2]:
from keras.layers import Input, Dense
from keras.models import Model
from keras import regularizers

/home/bruno/.virtualenvs/daily/local/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
# use Matplotlib (don't ask)
import matplotlib.pyplot as plt
%matplotlib inline

## Delta M - Delta t 

Now with the deltamdeltat diagrams

In [10]:
df_mira = pd.read_csv('ATLAS_LC/MIRA_features_table.csv')
signature_cols = [col for col in df_mira.columns if 'Deltam' in col]
#plt.imshow(df_mira[signature_cols].iloc[2787].as_matrix().reshape(23, 24).T, cmap='inferno')
#plt.show()

In [11]:
signature_cols += ['OBJID', 'filter', 'CLASS']
df_mira = df_mira[signature_cols]

In [12]:
df_mpulse = pd.read_csv('ATLAS_LC/MPULSE_features_table.csv')[signature_cols]
df_dbf = pd.read_csv('ATLAS_LC/DBF_features_table.csv')[signature_cols]
df_lpv = pd.read_csv('ATLAS_LC/LPV_features_table.csv')[signature_cols]
df_dbh = pd.read_csv('ATLAS_LC/DBH_features_table.csv')[signature_cols]
df_pulse = pd.read_csv('ATLAS_LC/PULSE_features_table.csv')[signature_cols]
df_nsine = pd.read_csv('ATLAS_LC/NSINE_features_table.csv')[signature_cols]
df_sine = pd.read_csv('ATLAS_LC/SINE_features_table.csv')[signature_cols]
df_msine = pd.read_csv('ATLAS_LC/MSINE_features_table.csv')[signature_cols]
df_cbh = pd.read_csv('ATLAS_LC/CBH_features_table.csv')[signature_cols]
df_cbf = pd.read_csv('ATLAS_LC/CBF_features_table.csv')[signature_cols]
df_irr = pd.read_csv('ATLAS_LC/IRR_features_table.csv')[signature_cols]

In [13]:
full_data = pd.concat([df_mira, df_mpulse, df_dbf, df_lpv, df_dbh, df_pulse, 
                       df_nsine, df_sine, df_msine, df_cbf, df_cbh])
signature_cols = [col for col in df_mira.columns if 'Deltam' in col]
X = full_data[signature_cols].as_matrix()

/home/bruno/.virtualenvs/daily/lib/python2.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.


In [14]:
del(df_mira)

In [15]:
del(df_mpulse)

In [16]:
del(df_dbf)

In [17]:
del(df_lpv)

In [18]:
del(df_dbh)

In [19]:
del(df_pulse)

In [20]:
del(df_cbf)

In [21]:
del(df_cbh)

In [22]:
del(df_nsine)

In [23]:
del(df_sine)

In [24]:
del(df_msine)

In [25]:
del(df_irr)

In [27]:
from sklearn.model_selection import train_test_split

In [28]:
X_train, X_test = train_test_split(X, test_size=0.10, random_state=17)

In [29]:
print X_train.shape
print X_test.shape

(326278, 552)
(36254, 552)


In [41]:
## this is the size of our encoded representations
encoding_dim = 64  # 32 floats -> compression of factor 24.5, assuming the input is 784 floats

# this is our input placeholder
input_img = Input(shape=(23*24,))
# "encoded" is the encoded representation of the input
encoded = Dense(encoding_dim, activity_regularizer=regularizers.l1(10e-5), activation='relu')(input_img)
# "decoded" is the lossy reconstruction of the input
decoded = Dense(23*24, activation='sigmoid')(encoded)

# this model maps an input to its reconstruction
autoencoder = Model(input_img, decoded)

NameError: name 'regularizers' is not defined

In [ ]:
# this model maps an input to its encoded representation
encoder = Model(input_img, encoded)

In [ ]:
# create a placeholder for an encoded (32-dimensional) input
encoded_input = Input(shape=(encoding_dim,))
# retrieve the last layer of the autoencoder model
decoder_layer = autoencoder.layers[-1]
# create the decoder model
decoder = Model(encoded_input, decoder_layer(encoded_input))

In [ ]:
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

In [ ]:
autoencoder.fit(X_train, X_train,
                epochs=50,
                batch_size=5000,
                shuffle=True,
                validation_data=(X_test, X_test))

autoencoder.fit(X_train, X_train,
                epochs=10,
                batch_size=9000,
                shuffle=True,
                validation_data=(X_test, X_test))

autoencoder.fit(X_train, X_train,
                epochs=20,
                batch_size=2500,
                shuffle=True,
                validation_data=(X_test, X_test))

autoencoder.fit(X_train, X_train,
                epochs=20,
                batch_size=5000,
                shuffle=True,
                validation_data=(X_test, X_test))


In [ ]:
# encode and decode some lightcurves
# note that we take them from the *test* set
encoded_imgs = encoder.predict(X_test)
decoded_imgs = decoder.predict(encoded_imgs)

In [ ]:
# use Matplotlib (don't ask)
import matplotlib.pyplot as plt

n = 6  # how many digits we will display
plt.figure(figsize=(12, 4))
for i in range(n):
    # display original
    ax = plt.subplot(2, n, i + 1)
    j = np.random.randint(0, high=len(X_test))
    plt.imshow(X_test[j].reshape(23, 24).T, cmap='inferno')
    
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_imgs[j].reshape(23, 24).T, cmap='inferno')
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

plt.tight_layout()

In [ ]:
# encode every data 
encoded_imgs = encoder.predict(X)

In [ ]:
encoded_imgs.shape

In [42]:
enc = pd.DataFrame()
enc['OBJID'] = full_data['OBJID']
enc['filter'] = full_data['filter']
enc['CLASS'] = full_data['CLASS']
for i in range(encoding_dim):
    enc['encode_{}'.format(i+1)] = encoded_imgs[:, i]
enc.to_csv('ATLAS_LC/deltamdeltat_{}_encoded.csv'.format(encoded.shape[1]))

In [ ]:
decoded_imgs = decoder.predict(encoded_imgs)
dec = pd.DataFrame()
dec['OBJID'] = full_data['OBJID']
dec['filter'] = full_data['filter']
dec['CLASS'] = full_data['CLASS']
for i in range(23*24):
    dec['encode_{}'.format(i+1)] = decoded_imgs[:, i]
dec.to_csv('ATLAS_LC/deltamdeltat_{}_decoded.csv'.format(decoded.shape[1]))